In [2]:
from get_datasets.Get_Datasets import get_data_array, get_datasets
from src.Evolutionary_Algorithm import create_next_population

import matplotlib.pyplot as plt
import numpy as np


In [4]:
x_train, y_train, x_val, y_val, x_test, y_test = get_data_array(train_dir='chest_xray_new/train',
                                                                test_dir='chest_xray_new/test')

  0%|          | 0/3883 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_ds, val_ds, test_ds = get_datasets(x_train, y_train, x_test, y_test, x_val, y_val)

In [ ]:
sample_images, _ = next(iter(train_ds))
plt.figure(figsize=(10, 10))
for i, image in enumerate(sample_images[:9]):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image.numpy().astype("int"))
    plt.axis("off")